In [7]:
import numpy as np 
import pandas as pd 
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout , Activation
from keras.layers import Conv2D, MaxPooling2D
import cv2,os
from keras.utils import np_utils

In [8]:
path_train = "../Learn facial expressions/train/"
path_test = "../Learn facial expressions/test/"

In [10]:
#load data train, thêm data , làm ảnh không mờ
def load_data(path):
    l=0
    for folder in os.listdir(path):
        path_folder = path + folder
        for name_image in os.listdir( path_folder):
            path_image = path_folder +"/"+ name_image
            image = cv2.imread(path_image,0)
            img_kernel = cv2.medianBlur(image,5)
            X_train.append(img_kernel)
            y_train.append(l)
            quadrant_turn = [15,30,45,90,135,180,270]
            if folder == "disgust":
                for j in quadrant_turn:
                    (h, w) = image.shape
                    center = (w / 2, h / 2)
                    check = cv2.getRotationMatrix2D(center, j , 0)
                    img_turn = cv2.warpAffine(image, check, (h, w))
                    img_kernel = cv2.medianBlur(img_turn,5)
                    X_train.append(img_kernel)
                    y_train.append(l)
        l+=1

In [11]:
load_data(path_train)

In [12]:
#load dât test
X_test, y_test =[],[]
for folder in os.listdir(path_test):
    l=0
    path_folder = path_test + folder
    for name_image in os.listdir( path_folder):
        path_image = path_folder +"/"+ name_image
        image = cv2.imread(path_image,0)
        img_kernel = cv2.medianBlur(image,5)
        X_test.append(img_kernel)
        y_test.append(l)
    l+=1

In [13]:
X_train = np.array(X_train) 
X_train = X_train/255
X_test = np.array(X_test) 
X_test= X_test/255
X_train= X_train.reshape(X_train.shape[0],48,48,1)
X_val = X_train
X_test = X_test.reshape(X_test.shape[0],48,48,1)
y_train =np.array(y_train)
y_val = np.array(y_train)
y_test = np.array(y_test)



In [14]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_val.shape)
print(y_val.shape)

(31761, 48, 48, 1)
(31761,)
(7178, 48, 48, 1)
(7178,)
(31761, 48, 48, 1)
(31761,)


In [17]:
from keras.models import load_model
loaded_model=load_model('facial_expresion_comparsion_model.hdf5')
predict=loaded_model.fit(X_test,y_test)


225/225 [==============================] - 166s 734ms/step - loss: 0.1023 - accuracy: 0.9855


In [9]:
#model
model = Sequential()

In [10]:
#box1
# dùng hàm rule làm activation và chỉ rõ input_shape cho layer đầu tiên
model.add(Conv2D(64,(3,3), activation = 'relu',input_shape=(48,48,1),padding='same', data_format='channels_last'))
model.add(Conv2D(64,(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

In [11]:
#box2
model.add(Conv2D(128,(3,3), activation = 'relu',padding='same', data_format='channels_last'))
model.add(Conv2D(128,(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


In [12]:
#box3
model.add(Conv2D(256,(3,3), activation = 'relu',padding='same', data_format='channels_last'))
model.add(Conv2D(256,(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

In [13]:
#box4
model.add(Conv2D(512,(3,3), activation = 'relu',padding='same', data_format='channels_last'))
model.add(Conv2D(512,(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

In [18]:
'''model.add(Conv2D(256,(7,7), activation = 'relu',padding='same', data_format='channels_last'))
model.add(Conv2D(256,(7,7), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))'''

model.add(Flatten()) #Flatten layer chuyển từ tensor sang vector
model.add(Dense(128, activation='relu'))# Thêm Fully Connected layer với 128 nodes và dùng hàm rule
model.add(Dense(10, activation='softmax'))# Output layer với 10 node và dùng sigmoid function để chuyển sang xác suất.

NameError: name 'model' is not defined

In [15]:
# đùng để tối ưu hàm loss function.
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [16]:
H = model.fit(X_train, y_train, validation_data= (X_val,y_val) ,batch_size=32, epochs=20, verbose=1)

Epoch 1/20
993/993 [==============================] - 790s 782ms/step - loss: 1.7589 - accuracy: 0.2911 - val_loss: 1.4298 - val_accuracy: 0.4397
Epoch 2/20
993/993 [==============================] - 824s 830ms/step - loss: 1.4208 - accuracy: 0.4463 - val_loss: 1.3003 - val_accuracy: 0.4977
Epoch 3/20
993/993 [==============================] - 821s 827ms/step - loss: 1.2830 - accuracy: 0.5041 - val_loss: 1.1808 - val_accuracy: 0.5477
Epoch 4/20
993/993 [==============================] - 823s 829ms/step - loss: 1.1888 - accuracy: 0.5490 - val_loss: 1.0283 - val_accuracy: 0.6176
Epoch 5/20
993/993 [==============================] - 822s 828ms/step - loss: 1.0474 - accuracy: 0.6032 - val_loss: 0.8816 - val_accuracy: 0.6817
Epoch 6/20
993/993 [==============================] - 821s 827ms/step - loss: 0.8946 - accuracy: 0.6646 - val_loss: 0.6975 - val_accuracy: 0.7534
Epoch 7/20
993/993 [==============================] - 821s 827ms/step - loss: 0.7046 - accuracy: 0.7361 - val_loss: 0.4920 -

In [17]:
score = model.evaluate(X_test, y_test, verbose=0)
print(score)

[9.450490951538086, 0.09974923729896545]


NameError: name 'X_test' is not defined